### Install the Fabric Data Agent SDK

In [1]:
# %pip install fabric-data-agent-sdk
%pip install -q /lakehouse/default/Files/fabric_data_agent_sdk-0.1.16a0-py3-none-any.whl

StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 8, Finished, Available, Finished)

Reason for being yanked: Yanked due to conflicts with CVE-2024-35195 mitigation
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires sentencepiece, which is not installed.
sentence-transformers 2.0.0 requires torchvision, which is not installed.
dash 2.14.0 requires Flask<2.3.0,>=1.0.4, but you have flask 3.0.0 which is incompatible.
dash 2.14.0 requires Werkzeug<2.3.0, but you have werkzeug 3.0.1 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.15.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



### Define environment variables

In [2]:
from datetime import datetime
from synapse.ml.pymds import get_mds_json_logger
import logging
import json
import sys
import time
import requests
import pandas as pd
import uuid
import os

from fabric.dataagent.client import (
    FabricOpenAI,
    FabricDataAgentManagement,
    create_data_agent,
    delete_data_agent,
)

env = sc.getConf().get("spark.trident.pbienv").lower()
if env == "prod":
    env = sc.getConf().get("spark.cluster.region").lower()

workspace_id = sc._jsc.hadoopConfiguration().get('trident.artifact.workspace.id')

# Look at runner artifacts_creation_da_runners for how the data sources are created
lakehouse_name = "DAlakehouse"

data_agent_name = f"DATA_AGENT_FEW_SHOT_VALIDATION"

StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 10, Finished, Available, Finished)

### Setup the Data Agent

In [3]:
def setup_data_agent():
    # Create Data Agent (handles both new creation and connecting to existing)
    data_agent = create_data_agent(data_agent_name)

    # Add lakehouse data source
    data_agent.add_datasource(lakehouse_name, workspace_id, "lakehouse")

    # Publish Data Agent
    data_agent.publish()

    return data_agent


StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 11, Finished, Available, Finished)

### Define few-shot examples for testing

In [4]:
# Create synthetic examples with intentional conflicts for demonstration
# These examples showcase different conflict patterns:
# - Same intent but different data sources (tables/views)
# - Same intent but different aggregation methods
# - Same intent but different granularity levels

examples_to_add = {
    "What was total revenue for Product Alpha in Q1 2024?": "SELECT SUM(amount) AS revenue FROM sales WHERE product = 'Alpha' AND fiscal_quarter = '2024-Q1';",
    "Show me average deal size in the North region during 2023.": "SELECT AVG(amount) AS avg_deal FROM deals WHERE region = 'North' AND YEAR(closed_date) = 2023;",
    "How many support tickets were closed in January 2024?": "SELECT COUNT(*) AS tickets_closed FROM support_tickets WHERE status = 'Closed' AND DATE_TRUNC('month', closed_at) = '2024-01-01';",
    "What is the total revenue for Product Alpha in the first quarter of 2024?": "SELECT COUNT(DISTINCT order_id) AS revenue FROM order_facts WHERE product = 'Alpha' AND fiscal_quarter = '2024-Q1';",
    "How many new leads were generated from the website in February 2024?": "SELECT COUNT(*) AS web_leads FROM leads WHERE source = 'Web' AND DATE_TRUNC('month', created_at) = '2024-02-01';",
    "List total marketing touches for campaign Ignite in March 2024.": "SELECT SUM(touches) AS total_touches FROM campaign_metrics WHERE campaign_name = 'Ignite' AND DATE_TRUNC('month', activity_date) = '2024-03-01';",
    "What was the average deal amount in the North region during 2023?": "SELECT SUM(amount) / COUNT(*) AS avg_deal FROM deal_summary WHERE region = 'North' AND YEAR(closed_date) = 2023;",
    "Which products exceeded 1M revenue in 2023?": "SELECT product FROM sales WHERE YEAR(order_date) = 2023 GROUP BY product HAVING SUM(amount) > 1000000;",
    "Show me how many support tickets were closed during January 2024.": "SELECT COUNT(ticket_id) AS tickets_closed FROM ticket_events WHERE event_type = 'Closed' AND MONTH(event_time) = 1 AND YEAR(event_time) = 2024;",
    "What is the churn rate for subscription tier Gold in 2024 so far?": "SELECT SUM(churned_accounts)::float / NULLIF(SUM(active_accounts), 0) AS churn_rate FROM subscription_health WHERE tier = 'Gold' AND YEAR(snapshot_date) = 2024;",
}


StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 12, Finished, Available, Finished)

# Execution

In [5]:
# Setup the data agent
data_agent = setup_data_agent()

# Get the datasource
datasources = data_agent.get_datasources()
if len(datasources) == 0:
    raise ValueError("No datasources found for the data agent")

datasource = datasources[0]

# Add the few-shot examples to the datasource
try:
    datasource.add_fewshots(examples_to_add)
    print(f"Added {len(examples_to_add)} few-shot examples to the datasource")
except Exception as e:
    print(f"Note: {e}")
    print("Few-shots may already exist in the datasource")


StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 13, Finished, Available, Finished)

Added 10 few-shot examples to the datasource


In [6]:
# Evaluate few-shot examples using the datasource's evaluate_few_shots method
result = datasource.evaluate_few_shots(batch_size=20)

# Print out the overall success rate
print(f"Success rate: {result.success_rate:.2f}% ({result.success_count}/{result.total_examples})")

# Display success and failure cases
print("\nSuccess Cases:")
success_df = result.success_cases
display(success_df)

print("\nFailure Cases:")
failure_df = result.failure_cases
display(failure_df)


StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 14, Finished, Available, Finished)

Quality-approved examples: 27/30 (filtered out 3 low-quality examples)
[conflict-usage] completion=1264
Success rate: 90.00% (27/30)

Success Cases:


SynapseWidget(Synapse.DataFrame, e1316f70-a8c5-4b5a-9a8b-4a0ba6b3fcca)


Failure Cases:


SynapseWidget(Synapse.DataFrame, b5b232bb-8a4b-4c61-bdcb-05656d7be100)

In [7]:
# Display conflict summary
print(f"\nConflicts Detected: {result.conflict_count}")
print(f"Confidence Ratings: 5=High, 4=Medium, 3=Low, 2=Very Low, 1=Speculative\n")

# Display detailed conflict information if any
if result.conflict_count > 0:
    print("Detailed Conflict Analysis:")
    conflict_details_df = result.conflict_details
    display(conflict_details_df)
else:
    print("No conflicts detected among quality-approved examples.")


StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 15, Finished, Available, Finished)


Conflicts Detected: 25
Confidence Ratings: 5=High, 4=Medium, 3=Low, 2=Very Low, 1=Speculative

Detailed Conflict Analysis:


SynapseWidget(Synapse.DataFrame, f7464aed-3c78-4437-b24f-7bdda7ff7500)

In [8]:
# Assert success rate meets threshold for pipeline validation
MINIMUM_SUCCESS_RATE = 50.0  # Adjust as needed for your pipeline requirements

if result.success_rate < MINIMUM_SUCCESS_RATE:
    raise AssertionError(f"Few-shot validation failed: Success rate {result.success_rate:.2f}% is below threshold {MINIMUM_SUCCESS_RATE}%")

print(f"✓ Few-shot validation passed: Success rate {result.success_rate:.2f}% meets threshold {MINIMUM_SUCCESS_RATE}%")


StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 16, Finished, Available, Finished)

✓ Few-shot validation passed: Success rate 90.00% meets threshold 50.0%


In [9]:
# Clean up - delete the data agent
delete_data_agent(data_agent_name)
print(f"Data agent '{data_agent_name}' deleted successfully")

StatementMeta(, c80e67a6-ab9c-491d-a6de-103802b06e20, 17, Finished, Available, Finished)

Data agent 'DATA_AGENT_FEW_SHOT_VALIDATION' deleted successfully
